# UTOPIA example notebook

In [ ]:
import enum

import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import tqdm

import byotrack
import byotrack.icy.io
from byotrack.implementation.refiner.interpolater import ForwardBackwardInterpolater
import byotrack.visualize

import schya.detrending
import schya.extraction
import schya.filtering
import schya.spike
import schya.visualize

TEST = False  # Will reduce the size of the video to play with

## Loading videos

In [ ]:
gcamp_path = "path/to/gcamp_video"
tdtomato_path = "path/to_tdtomato/video"

gcamp_video = byotrack.Video(gcamp_path)
tdtomato_video = byotrack.Video(tdtomato_path)

if TEST:
    gcamp_video = gcamp_video[:100]
    tdtomato_video = tdtomato_video[:100]

# Normalize videos into [0, 1]
gcamp_video.set_transform(byotrack.VideoTransformConfig(aggregate=True, normalize=True, q_min=0.02, q_max=0.995, smooth_clip=1.0))
tdtomato_video.set_transform(byotrack.VideoTransformConfig(aggregate=True, normalize=True, q_min=0.02, q_max=0.995, smooth_clip=1.0))

In [ ]:
# Display the video
# Use w/x to move forward in time (or space to run/pause the video)
# Use v to display none, green, red or both channels

schya.visualize.TwoColorInteractiveVisualizer((tdtomato_video, gcamp_video)).run()

## Track TDTomato

### Loading saved tracks

Prevents to rerun the tracking each time

In [ ]:
# Reload tracks saved in the byotrack format

tracks = byotrack.Track.load("tracks.pt")

In [ ]:
# Or reload tracks from Icy xml format

tracks = byotrack.icy.io.load_tracks("tracks_tdTomato.xml")

### Tracking pipeline

In [ ]:
from byotrack.implementation.detector.stardist import StarDistDetector
from byotrack.implementation.linker.icy_emht import IcyEMHTLinker
from byotrack.implementation.refiner.cleaner import Cleaner
from byotrack.implementation.refiner.stitching.emc2 import EMC2Stitcher

#### Detections

In [ ]:
model_path = "path/to/model_folder"

detector = StarDistDetector(model_path, batch_size=1)

In [ ]:
detections_sequence = detector.run(tdtomato_video)

In [ ]:
# Display the detections with opencv
# Use w/x to move forward in time (or space to run/pause the video)
# Use v to switch on/off the display of the video
# Use d to switch detection display mode (None, mask, segmentation)

byotrack.visualize.InteractiveVisualizer(tdtomato_video, detections_sequence).run()

#### Build tracklets

In [ ]:
# Run linking

icy_path = "path/to/icy.jar"

linker = IcyEMHTLinker(icy_path)
linker.motion = linker.Motion.BROWNIAN
tracklets = linker.run(tdtomato_video, detections_sequence)

In [ ]:
# Visualize life span

byotrack.visualize.display_lifetime(tracklets)

#### Tracklet stitching

In [ ]:
# Cleaning + EMC2

cleaner = Cleaner(min_length=5, max_dist=3.5)
tracks = cleaner.run(tdtomato_video, tracklets)

stitcher = EMC2Stitcher(eta=5.0)  # Don't link tracks if they are too far (EMC dist > 5 (pixels))
tracks = stitcher.run(tdtomato_video, tracks)

In [ ]:
# Visualize new life span

byotrack.visualize.display_lifetime(tracks)

In [ ]:
# Save tracks

byotrack.Track.save(tracks, "tracks.pt")  # Can be reload with byotrack.Track.load("tracks.pt")

### Tracks visualization

You can export track to icy format and visualize them with icy, or use our own tool of visualization (or build new ones in python)

In [ ]:
# Export track to icy
# Needs to fill hole in tracks before saving with Forward backward interpolator

interpolater = ForwardBackwardInterpolater(method="constant", full=False)

byotrack.icy.io.save_tracks(interpolater.run(tdtomato_video, tracks), "test.xml")

In [ ]:
byotrack.visualize.display_lifetime(tracks)

In [ ]:
# Display the tracks with opencv
# Use w/x to move forward in time (or space to run/pause the video)
# Use v (resp. t) to switch on/off the display of video (resp. tracks)

# You can also give the detections_sequence object to the visualizer
# Then, use d to switch detection display mode (None, mask, segmentation)

byotrack.visualize.InteractiveVisualizer(tdtomato_video, tracks=tracks).run()

## Calcium signal extraction

### Select long tracks and complete them

In [ ]:
from byotrack.implementation.refiner.interpolater import ForwardBackwardInterpolater

# keep only big enough tracks (Cover at least 80% of video from start to end)

valid_tracks = [len(t) > 0.80 * len(tdtomato_video) for t in tracks]

interpolater = ForwardBackwardInterpolater(method="tps", full = True, alpha=10.0)
final_tracks = interpolater.run(tdtomato_video, tracks)  # Interpolate using all tracks, and filter afterwards
final_tracks = [track for i, track in enumerate(final_tracks) if valid_tracks[i]]

print(f"Kept {len(final_tracks)} valid tracks from {len(tracks)} tracks")

### TdTomato control intensities extraction

In [ ]:
# Extract control intensities from tdtomato sequence

ctrl_intensities = schya.extraction.extract_intensities_from_roi(tdtomato_video, final_tracks, 9)

### Extract GCsub roi tracking

In [ ]:
raw_intensities, calcium_positions = schya.extraction.SubRoiExtractor(
    gcamp_video, 
    byotrack.Track.tensorize(final_tracks).numpy(),
    25,  # Roi size (25 x 25)
    4,  # Max relative motion between two consecutive frames
).compute()

In [ ]:
# Build the calcium tracks for visualization

calcium_tracks = []

for k in range(0, len(final_tracks)):
    calcium_tracks.append(byotrack.Track(0, torch.tensor(calcium_positions[:, k]).to(torch.float32), final_tracks[k].identifier))

In [ ]:
# Visualize the calcium tracks
# Use w/x to move forward in time (or space to run/pause the video)
# Use t to display none, green, red or both tracks
# Use v to display none, green, red or both channels

vis = schya.visualize.TwoColorInteractiveVisualizer((tdtomato_video, gcamp_video), tracks=final_tracks, calcium_tracks=calcium_tracks)

vis.scale = 1  # Increase/decrease the size of the display
vis._display_video = 3  # GCaMP

vis.run()

In [ ]:
# Smoothing (Reduce smoothing by reducing the window size (=> 1 = no smoothing))

window_size = 9

smoothed_intensities = schya.extraction.smooth(raw_intensities, window_size)
ctrl_smoothed_intensities = schya.extraction.smooth(ctrl_intensities, window_size)

In [ ]:
# Example of track 50 which is pretty nice + 16 others
# You can select another t_id or batch_id

t_id = 50  # 14, 50 are nice examples
batch_id = 0

plt.plot(raw_intensities[t_id], label="Raw")
plt.plot(smoothed_intensities[t_id], label="Smoothed")
plt.plot(ctrl_intensities[t_id], label="Ctrl")
plt.plot(ctrl_smoothed_intensities[t_id], label="Ctrl Smoothed")
plt.legend()
plt.show()


fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))

for i in range(4):
    for j in range(4):
        k = batch_id * 16 + i * 4 + j
        axs[i, j].plot(raw_intensities[k % len(raw_intensities)])
        axs[i, j].plot(smoothed_intensities[k % len(raw_intensities)])
        axs[i, j].plot(ctrl_intensities[k % len(raw_intensities)])
        axs[i, j].plot(ctrl_smoothed_intensities[k % len(raw_intensities)])

plt.show()

### Detrending

In [ ]:
corrected_intensities = schya.detrending.ica_decorr(smoothed_intensities, ctrl_smoothed_intensities, 0.5, 10)

In [ ]:
# Additional independent detrending -> either polynomial fit or Frequency filtering to remove the baseline

if False:
    detrended_intensities = schya.detrending.detrend_all(corrected_intensities, 17)
else:
    # Drop baseline (period larger than 100 frames)
    detrended_intensities = schya.detrending.high_pass_filter(corrected_intensities, 1 / 100)

In [ ]:
# Example of track 50 which is pretty nice + 16 others
# You can select another t_id or batch_id

t_id = t_id
batch_id = 0

plt.plot(corrected_intensities[t_id], label="ICA corrected")
plt.plot(detrended_intensities[t_id], label="Detrended")
#plt.plot(smoothed_intensities[t_id] * 10, label="Intensity (x10)")
#plt.plot(ctrl_smoothed_intensities[t_id] * 10, label="Control (x10)")
plt.legend()
plt.show()


fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))

for i in range(4):
    for j in range(4):
        k = batch_id * 16 + i * 4 + j
        axs[i, j].plot(corrected_intensities[k % len(raw_intensities)])
        axs[i, j].plot(detrended_intensities[k % len(raw_intensities)])

plt.show()

### Filtering

In [ ]:
is_noise = schya.filtering.is_noise(detrended_intensities, 1e-5)
print(f"Found {is_noise.sum()} noise signals")

In [ ]:
noise = detrended_intensities[is_noise]
neurons = detrended_intensities[~is_noise]

# Propagate t_id
t_id = np.arange(len(neurons))[np.arange(is_noise.shape[0])[~is_noise] == t_id][0]

In [ ]:
# Let's see a batch of neurons vs a batch of noise
# You can select another batch_id

batch_id = 0

fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))

for i in range(4):
    for j in range(4):
        k = batch_id * 16 + i * 4 + j
        axs[i, j].plot(neurons[k % len(neurons)])
        axs[i, j].plot(noise[k % len(noise)] - 5)

plt.show()

### Spike extraction

In [ ]:
# Apply foopsi to extract spikes and calcium signal reconstruction

calcium_reconstruction, spikes = schya.spike.foopsi_all(neurons)

In [ ]:
# Clusterize spikes
# You can use different std (how far the kernel will look for neighboring spikes to aggregate with the current one)
# 5 yields pretty good results

true_spikes = schya.spike.clusterize_spikes(spikes, std=5.0)

In [ ]:
# Example to see how well the clustering has worked

plt.plot(spikes[0], label="Spikes")
plt.plot(true_spikes[0] + 0.25, label="Clustered")
plt.legend()
plt.show()

In [ ]:
# Visualize same neuron as before + 16 others
# You can select another t_id or batch_id

t_id = t_id
batch_id = 0

plt.plot(neurons[t_id], label="Calcium signal")
plt.plot(calcium_reconstruction[t_id], label="Reconstruction")
plt.plot(spikes[t_id] * 7, label="All spikes")
plt.plot(true_spikes[t_id] * 7, label="Clustered spikes")
plt.legend()
plt.show()


fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))

for i in range(4):
    for j in range(4):
        k = batch_id * 16 + i * 4 + j
        axs[i, j].plot(neurons[k % len(neurons)])
        axs[i, j].plot(calcium_reconstruction[k % len(neurons)])
        axs[i, j].plot(spikes[k % len(neurons)] * 7)
        axs[i, j].plot(true_spikes[k % len(neurons)] * 7)

plt.show()

In [ ]:
# Filter low spikes to keep only meaningful ones

# First plot all spikes
plt.title("All spikes")
plt.xlabel("Frames")
plt.ylabel("Neurons")
binarized_spikes = spikes > 0
pos = schya.spike.to_raster_pos(binarized_spikes)
plt.eventplot(pos)
plt.show()


# First plot all spikes
plt.title("Kept spikes")
plt.xlabel("Frames")
plt.ylabel("Neurons")
binarized_spikes = schya.spike.binarize_ratio(true_spikes, 0.2)
pos = schya.spike.to_raster_pos(binarized_spikes)
plt.eventplot(pos)
plt.show()

In [ ]:
# Visualize same neuron as before + 16 others
# You can select another t_id or batch_id

t_id = t_id
batch_id = 0

plt.plot(raw_intensities[~is_noise][t_id] * 5, label="Raw intensities (*5)")
plt.plot(neurons[t_id], label="Calcium signal")
plt.plot(binarized_spikes[t_id] - 1, label="Selected spikes")
plt.legend()
plt.show()


fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))

for i in range(4):
    for j in range(4):
        k = batch_id * 16 + i * 4 + j
        axs[i, j].plot(raw_intensities[~is_noise][k % len(neurons)] * 5)
        axs[i, j].plot(neurons[k % len(neurons)])
        axs[i, j].plot(binarized_spikes[k % len(neurons)] - 1)

plt.show()